In [58]:
import pandas as pd

customer_data = pd.read_csv('data/customer_data.csv')
customer_data.head()
#todo add business name eg James William Agriculutre - later


,Customer Name,uid,business sector,location,phone number,national ID,passport,tin,gender,education level,business start date
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,256723196874,78602628,U6482628,8900454,Female,University,2013-02-25
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,256735895263,36028682,U1006980,7442154,Female,University,2009-02-22
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,256790291092,22644880,U3152513,2102298,Male,Vocational,2009-02-07
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,256773437304,78654217,U3734663,9738952,Male,University,2016-04-15
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,256712698533,71758895,U4129268,4037967,Male,Primary,2013-03-15


In [78]:
transactions_data = pd.read_csv('data/transactions_data.csv')
transactions_data.head()

,Customer Name,uid,amount,type,reason,date
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,70932.51,Debit,Refund,2023-07-15
1,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,630576.83,Credit,Sale,2019-01-15
2,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,984729.90,Credit,Purchase,2020-05-17
3,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,456349.56,Credit,Sale,2022-09-30
4,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,754581.57,Credit,Miscellaneous,2023-03-04


In [60]:
print(f"Number of rows in transactions_data: {transactions_data.shape[0]}")

Number of rows in transactions_data: 89695


# Data cleaning
- Remove columns from customer_data df which are not needed for our analysis
- Add a new column to customer_data which shows the business duration
- Add new columsn which calculate the value: some value containng business age and number and amount of transactions (maybe business_score?)

In [61]:

customer_data_cleaned = customer_data.drop(columns=['phone number', 'national ID', 'passport', 'tin'])
customer_data_cleaned.head()

,Customer Name,uid,business sector,location,gender,education level,business start date
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,Female,University,2013-02-25
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,Female,University,2009-02-22
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,Male,Vocational,2009-02-07
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,Male,University,2016-04-15
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,Male,Primary,2013-03-15


Add business duration column

In [66]:
from datetime import datetime

#convert the business start date to data time so that we can calculate year
customer_data['business start date'] = pd.to_datetime(customer_data['business start date'])

duration_in_days = (datetime.now() - customer_data['business start date']).dt.days

duration_in_years = (duration_in_days / 365).round(2)

customer_data_cleaned['business duration (years)'] = duration_in_years

customer_data_cleaned.head()


,Customer Name,uid,business sector,location,gender,education level,business start date,business duration (years)
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,Female,University,2013-02-25,10.57
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,Female,University,2009-02-22,14.58
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,Male,Vocational,2009-02-07,14.62
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,Male,University,2016-04-15,7.43
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,Male,Primary,2013-03-15,10.52


### Add performance index column

Custom formula for *perfomance index*:

Perfomance Index =  (λ1 * Net Profit Per Year +  λ2 * Lifetime Net Profit + λ3 * Transaction Volume Per Year)


Where:

- λ1 is the weight for Net Profit Per Year
- λ2 is the weight for Transaction Volume Per Year
- λ3 is the weight for Lifetime Net Profit


In [84]:
# Calculate Net Profit
total_credit = transactions_data[transactions_data['type'] == 'Credit'].groupby('uid')['amount'].sum()
total_debit = transactions_data[transactions_data['type'] == 'Debit'].groupby('uid')['amount'].sum()
net_profit = total_credit.subtract(total_debit, fill_value=0)

# Calculate Transaction Volume
transaction_volume = transactions_data.groupby('uid').size()

# Merge net profit and transaction volume with customer_data on uid
customer_data_cleaned_with_net_profit = customer_data_cleaned.merge(net_profit.rename('Net Profit'), on='uid', how='left').fillna(0)
customer_data_cleaned_with_txn_vol = customer_data_cleaned_with_net_profit.merge(transaction_volume.rename('Transaction Volume'), on='uid', how='left').fillna(0)

# Calculate net profit per year and transaction volume per year
customer_data_cleaned_with_txn_vol['Net Profit Per Year'] = customer_data_cleaned_with_txn_vol['Net Profit'] / customer_data_cleaned_with_txn_vol['business duration (years)']
customer_data_cleaned_with_txn_vol['Transaction Volume Per Year'] = customer_data_cleaned_with_txn_vol['Transaction Volume'] / customer_data_cleaned_with_txn_vol['business duration (years)']

# Weights
λ1, λ2, λ3 = 1, 1, 1

# Calculate perfomance index considering lifetime profits
customer_data_cleaned_with_txn_vol['performance index'] = (λ1 * customer_data_cleaned_with_txn_vol['Net Profit Per Year'] + 
                                  λ2 * customer_data_cleaned_with_txn_vol['Transaction Volume Per Year'] + 
                                  λ3 * customer_data_cleaned_with_txn_vol['Net Profit'])

# Normalize the perfomance index: 
min_val = customer_data_cleaned_with_txn_vol['performance index'].min()
max_val = customer_data_cleaned_with_txn_vol['performance index'].max()
customer_data_cleaned_with_txn_vol['Normalized Performance Index'] = (customer_data_cleaned_with_txn_vol['performance index'] - min_val) / (max_val - min_val)

customer_data_cleaned_with_txn_vol.head()


,Customer Name,uid,business sector,location,gender,education level,business start date,business duration (years),Net Profit,Transaction Volume,Net Profit Per Year,Transaction Volume Per Year,performance index,Normalized Performance Index
0,James William,cca67301-ef3c-4496-b8fd-31847b7edd88,Agriculture,Fort Portal,Female,University,2013-02-25,10.57,3513483.23,294,3.324014e+05,27.814570,3.845912e+06,0.047195
1,James Nansukusa,7a1c5730-b8e7-4774-8746-90a317b73a82,Services,Kampala,Female,University,2009-02-22,14.58,-3204307.06,107,-2.197741e+05,7.338820,-3.424074e+06,0.042561
2,Nansukusa Sarah,fb94fff4-9217-470f-9e80-12621dea6191,Fisheries,Lira,Male,Vocational,2009-02-07,14.62,-3393780.01,265,-2.321327e+05,18.125855,-3.625895e+06,0.042432
3,Bridget Namwalwa,ff0563dc-6006-4b4f-a88a-1e7b64adb040,Manufacturing,Kisoro,Male,University,2016-04-15,7.43,-8005741.43,262,-1.077489e+06,35.262450,-9.083195e+06,0.038954
4,William William,a185cde9-4893-42c0-bcad-fbcf101d2a77,Hospitality,Kampala,Male,Primary,2013-03-15,10.52,-8434974.90,260,-8.018037e+05,24.714829,-9.236754e+06,0.038856


# Encoding
Transforming data into machine readable numbers to facilitate the analysis process. 

## Define value we want encode
Business Sector, Location, Gender, Education level, business age, some value containng business age and number and amount of transactions

The features we choose should be abe to describe groups of rows (as compated to geatures like phone number)

## Creating dummy varialbes/ encoding
Creating numerical values from categorical features can be approached in several ways. 
The easiest is assigning a value to each value of a features. e.g 
 Business Sector
    - Agriculture : 1
    - Service : 2
    - Fisheries : 3
However, this is not effecient for clustering algorithms as it will give inaccurate clusters

A better approach is the use of one-hot encoding, where a value is 1 and only 1 possibilty, while the rest are 0
